In [ ]:
!pip install unidecode -q
!pip install contractions
!pip install nltk
!pip install gensim

In [61]:
import pandas as pd
import numpy as np
import logging
import re
import multiprocessing
from time import time 
from nltk.corpus import stopwords
import contractions
from unidecode import unidecode
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
df_ = pd.read_csv('musk.csv', index_col=0)
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
df_

,type,author,text,created
0,Comment,UNCwesRPh,I had been on twitter prior to the musk takeov...,2023-01-30 18:49:31+00:00
2,Comment,wgp3,That article does not say what you imply at al...,2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,The OG musk duck lives on my wall.,2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,How dare he speak that way to the great and po...,2023-01-30 18:48:26+00:00
5,Comment,Copykill,Can’t wait to finally have an excuse not to sh...,2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"Like brain dead piranhas lmao, anything barely...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,What am I lying about? What's my agenda? Pleas...,2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,Hard disagree. I don't think he's a parody of ...,2023-05-01 07:07:36+00:00
85224,Comment,Viperions,Yeah. I think too many things are lining up ri...,2023-05-01 07:07:15+00:00


In [62]:
df_['type'].unique()

array(['Comment', '2023-04-30 15:47:51+00:00'], dtype=object)

In [63]:
# Remove column with only one value
df_.drop(columns=['type'])

,author,text,created
0,UNCwesRPh,I had been on twitter prior to the musk takeov...,2023-01-30 18:49:31+00:00
2,wgp3,That article does not say what you imply at al...,2023-01-30 18:49:28+00:00
3,HighAndDrunk,The OG musk duck lives on my wall.,2023-01-30 18:48:43+00:00
4,Louismaxwell23,How dare he speak that way to the great and po...,2023-01-30 18:48:26+00:00
5,Copykill,Can’t wait to finally have an excuse not to sh...,2023-01-30 18:48:10+00:00
...,...,...,...
85215,Emble12,"Like brain dead piranhas lmao, anything barely...",2023-05-01 07:09:53+00:00
85221,Pizza_in_Space,What am I lying about? What's my agenda? Pleas...,2023-05-01 07:08:43+00:00
85223,Da1realBigA,Hard disagree. I don't think he's a parody of ...,2023-05-01 07:07:36+00:00
85224,Viperions,Yeah. I think too many things are lining up ri...,2023-05-01 07:07:15+00:00


In [64]:
def remove_duplicates_and_na(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove duplicates and NA values'''
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    return df

In [65]:
def lower(text:str) -> str:
    '''Lower all characters'''
    return text.lower()

In [66]:
def replace_contractions(text:str) -> str:
    '''Replace contractions in string of text'''
    return contractions.fix(text)

In [67]:
def remove_url(text:str) -> str:
   '''Remove URLs'''
   return re.sub(r"((www.[^\s]+)|(https?://[^\s]+))", " ", text)

In [68]:
def replace_to_unicode(text:str) -> str:
    '''Replace special characters to unicode standard'''
    return unidecode(text)

In [69]:
def text_to_word_list(text:str) -> str:
    ''' Pre process and convert texts to a list of words '''
    text = str(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"'s ", " is ", text)
    text = re.sub(r"'ve ", " have ", text)
    text = re.sub(r"n't ", " not ", text)
    text = re.sub(r"'re ", " are ", text)
    text = re.sub(r"'d ", " would ", text)
    text = re.sub(r"'ll ", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"/r/", " subreddit ", text)
    text = re.sub(r"0 0", "00", text)
    text = re.sub(r";", " ; ", text)
    text = text.split()

    return text

In [72]:
df = df_.copy()
df = remove_duplicates_and_na(df)
df['text'] = df['text'].apply(lambda x: lower(x))
df['text'] = df['text'].apply(lambda x: remove_url(x))
df['text'] = df['text'].apply(lambda x: replace_contractions(x))
df['text'] = df['text'].apply(lambda x: replace_to_unicode(x))
df['text'] = df['text'].apply(lambda x: lower(x))
df['text'] = df['text'].apply(lambda x: text_to_word_list(x))


In [73]:
df

,type,author,text,created
0,Comment,UNCwesRPh,"[i, had, been, on, twitter, prior, to, the, mu...",2023-01-30 18:49:31+00:00
2,Comment,wgp3,"[that, article, does, not, say, what, you, imp...",2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,"[the, og, musk, duck, lives, on, my, wall]",2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,"[how, dare, he, speak, that, way, to, the, gre...",2023-01-30 18:48:26+00:00
5,Comment,Copykill,"[cannot, wait, to, finally, have, an, excuse, ...",2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"[like, brain, dead, piranhas, lmao, anything, ...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,"[what, am, i, lying, about, what, is, my, agen...",2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,"[hard, disagree, i, do, not, think, he, is, a,...",2023-05-01 07:07:36+00:00
85224,Comment,Viperions,"[yeah, i, think, too, many, things, are, linin...",2023-05-01 07:07:15+00:00


In [74]:
def remove_stop_words(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove stop words'''
    stops = set(stopwords.words('english'))
    df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stops])
    return df

In [75]:

df = remove_stop_words(df)
df = df[df.text.str.len() > 1]

In [76]:
df

,type,author,text,created
0,Comment,UNCwesRPh,"[twitter, prior, musk, takeover, talking, dire...",2023-01-30 18:49:31+00:00
2,Comment,wgp3,"[article, say, imply, states, feature, turns, ...",2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,"[og, musk, duck, lives, wall]",2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,"[dare, speak, way, great, powerful, musk, obvi...",2023-01-30 18:48:26+00:00
5,Comment,Copykill,"[cannot, wait, finally, excuse, shower, douche...",2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"[like, brain, dead, piranhas, lmao, anything, ...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,"[lying, agenda, please, correct, errors, anyth...",2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,"[hard, disagree, think, parody, elon, musk, su...",2023-05-01 07:07:36+00:00
85224,Comment,Viperions,"[yeah, think, many, things, lining, right, sma...",2023-05-01 07:07:15+00:00


In [77]:
sent = [row for row in df.text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 16:57:06: collecting all words and their counts
INFO - 16:57:06: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 16:57:08: PROGRESS: at sentence #50000, processed 1986720 words and 1229758 word types
INFO - 16:57:09: collected 1455363 token types (unigram + bigrams) from a corpus of 2450943 words and 61282 sentences
INFO - 16:57:09: merged Phrases<1455363 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 16:57:09: Phrases lifecycle event {'msg': 'built Phrases<1455363 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000> in 2.14s', 'datetime': '2023-05-20T16:57:09.005975', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'created'}
INFO - 16:57:09: exporting phrases from Phrases<1455363 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 16:57:10: FrozenPhrases lifecycle event {'msg': 'exported Frozen

['article',
 'say',
 'imply',
 'states',
 'feature',
 'turns',
 'crashes',
 'tesla',
 'admits',
 'default_behavior',
 'never',
 'tried_hide',
 'hence',
 'whole',
 'counting',
 'crash',
 'disengaged',
 'within',
 '5_seconds',
 'crash',
 'article',
 'say',
 'brings',
 'question',
 'times',
 'musk',
 'said',
 'feature',
 'give',
 'proof',
 'times',
 'musk',
 'said',
 'fact',
 'referring',
 'crashes',
 'turned',
 'right',
 'crash',
 'speculation',
 'best']

# Can separate files here

In [78]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 16:57:10: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2023-05-20T16:57:10.841099', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'created'}
INFO - 16:57:10: collecting all words and their counts
INFO - 16:57:10: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:57:12: PROGRESS: at sentence #50000, processed 1634433 words, keeping 173136 word types
INFO - 16:57:12: collected 190207 word types from a corpus of 2016887 raw words and 61282 sentences
INFO - 16:57:12: Creating a fresh vocabulary
INFO - 16:57:12: Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 70489 unique words (37.06% of original 190207, drops 119718)', 'datetime': '2023-05-20T16:57:12.427662', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0

Time to build vocab: 0.04 mins


In [79]:
start = time()

w2v_model.train(sentences,
                total_examples=w2v_model.corpus_count,
                epochs=30,
                report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 16:57:12: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 70489 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4 shrink_windows=True', 'datetime': '2023-05-20T16:57:12.980528', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'train'}
INFO - 16:57:14: EPOCH 0 - PROGRESS: at 32.84% examples, 276828 words/s, in_qsize 12, out_qsize 2
INFO - 16:57:15: EPOCH 0 - PROGRESS: at 73.08% examples, 302822 words/s, in_qsize 11, out_qsize 2
INFO - 16:57:15: EPOCH 0: training on 2016887 raw words (858260 effective words) took 2.6s, 325924 effective words/s
INFO - 16:57:16: EPOCH 1 - PROGRESS: at 33.36% examples, 261017 words/s, in_qsize 5, out_qsize 9
INFO - 16:57:17: EPOCH 1 - PROGRESS: at 75.05% examples, 299822 words/s, in_qsize 12, out_qsize 1
INFO - 16:57:18: EPOCH 1: training on 2016887 raw words (858933 effective words) 

Time to train the model: 1.35 mins


In [80]:
w2v_model.save("word2vec.model")

INFO - 16:58:33: Word2Vec lifecycle event {'fname_or_handle': 'word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-20T16:58:33.993721', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'saving'}
INFO - 16:58:33: storing np array 'vectors' to word2vec.model.wv.vectors.npy
INFO - 16:58:34: storing np array 'syn1neg' to word2vec.model.syn1neg.npy
INFO - 16:58:34: not storing attribute cum_table
INFO - 16:58:34: saved word2vec.model


In [81]:
file_export = df.copy()
file_export['old_text'] = file_export.text
file_export.old_text = file_export.old_text.str.join(' ')
file_export.text = file_export.text.apply(lambda x: ' '.join(bigram[x]))
file_export.to_csv('cleaned_dataset.csv', index=False)

In [82]:
from sklearn.cluster import KMeans
word_vectors = Word2Vec.load("word2vec.model").wv

model = KMeans(n_clusters=2,
               max_iter=1000,
               random_state=True,
               n_init=50)
model.fit(X=word_vectors.vectors.astype('double'))

INFO - 16:58:36: loading Word2Vec object from word2vec.model
INFO - 16:58:36: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 16:58:36: loading vectors from word2vec.model.wv.vectors.npy with mmap=None
INFO - 16:58:36: loading syn1neg from word2vec.model.syn1neg.npy with mmap=None
INFO - 16:58:36: setting ignored attribute cum_table to None
INFO - 16:58:36: Word2Vec lifecycle event {'fname': 'word2vec.model', 'datetime': '2023-05-20T16:58:36.386321', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'loaded'}


KMeans(max_iter=1000, n_clusters=2, n_init=50, random_state=True)

In [83]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('goalposts', 0.9980942010879517),
 ('summary_video', 0.9979171752929688),
 ('exact_conversation', 0.9972283840179443),
 ('level_scrutiny', 0.9970115423202515),
 ('monumentally_stupid', 0.9969017505645752),
 ('valid_argument', 0.9968544244766235),
 ('tim_address', 0.9968184232711792),
 ('true_altruism', 0.996793806552887),
 ('massively_overpriced', 0.9967292547225952),
 ('revenue_generation', 0.996704638004303)]

In [84]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [85]:
words = pd.DataFrame(word_vectors.index_to_key)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [86]:
words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,musk,"[-0.07508382, -0.031636886, 0.017578892, -0.02...",1,1,0.978675,0.978675
1,-,"[-0.050290406, 0.028971367, 0.05361907, 0.0202...",0,-1,0.841271,-0.841271
2,elon_musk,"[0.03359735, 0.004619063, 0.0552491, -0.006509...",1,1,1.081044,1.081044
3,people,"[-0.017200358, 0.018315213, -0.0030343845, 0.0...",1,1,1.355219,1.355219
4,like,"[-0.11874396, -0.043377627, 0.00095364836, 0.0...",1,1,1.305437,1.305437
...,...,...,...,...,...,...
70484,filth_water,"[-0.09895438, -0.023136625, 0.02333415, 0.0712...",0,-1,3.118367,-3.118367
70485,humiliation_anal,"[-0.09619634, -0.028401533, 0.020558404, 0.064...",0,-1,2.367644,-2.367644
70486,alpha_female,"[-0.0903291, -0.020118501, 0.035740037, 0.0691...",0,-1,2.945927,-2.945927
70487,degraded_compared,"[-0.08731098, -0.015690053, 0.037888546, 0.071...",0,-1,3.417990,-3.417990


In [87]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [88]:
def remove_not_in_dictionary(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove stop words'''
    stops = set(words.words)
    df['text'] = df['text'].apply(lambda x: [item for item in x if item in stops])
    return df

In [89]:
df = remove_not_in_dictionary(df)

/var/folders/dn/6362h3w12mqfb_jh4vg90vp40000gn/T/ipykernel_45371/2479509448.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: [item for item in x if item in stops])


In [90]:
file_export = df.copy()
file_export['old_text'] = file_export.text
file_export.old_text = file_export.old_text.str.join(' ')
file_export.text = file_export.text.apply(lambda x: ' '.join(bigram[x]))
file_export.to_csv('cleaned_dataset.csv', index=False)

In [91]:
df

,type,author,text,created
0,Comment,UNCwesRPh,"[twitter, prior, musk, takeover, talking, dire...",2023-01-30 18:49:31+00:00
2,Comment,wgp3,"[article, say, imply, states, feature, turns, ...",2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,"[og, musk, duck, lives, wall]",2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,"[dare, speak, way, great, powerful, musk, obvi...",2023-01-30 18:48:26+00:00
5,Comment,Copykill,"[cannot, wait, finally, excuse, shower, douche]",2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"[like, brain, dead, lmao, anything, barely, re...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,"[lying, agenda, please, correct, errors, anyth...",2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,"[hard, disagree, think, parody, elon, musk, su...",2023-05-01 07:07:36+00:00
85224,Comment,Viperions,"[yeah, think, many, things, lining, right, sma...",2023-05-01 07:07:15+00:00
